## Function Calling与跨模型协作

## 💡 这节课会带给你

1. 理解Function Calling的概念
2. 理解Function Calling的工作原理
3. 实战使用OpenAI提供的Function Calling接口（基础请求及优化）
4. 探讨自定义Function的提供的可能性
5. 探讨Function Calling在大模型应用场景中带来的“质变”

## 学习能力要求

- 代码能力要求：中/低
- AI/数学基础要求：无
- 学习方式
    - 开发能力较强的同学：关注概念、工作原理、实操落地方法
    - 开发能力较弱的同学：关注概念、工作原理，理解Function Calling带来的可能性

## Function Calling的概念

Function Calling（函数调用），顾名思义，为模型提供了一种调用函数的方法/能力。

- Function Calling成立的模型能力基础：
    - 问题理解和行动规划
    - 结构化数据输出
    - In-Context Learning

Function Calling让模型输出不再局限于自身推理输出，而是可以与外部系统交互，完成更复杂的任务

- 常见Function Calling应用场景包括：
    - 查询检索，补充额外信息（如RAG、搜索）
    - 理解用户输入，向外部系统写入信息（如表单填写）
    - 调用外部系统能力，完成实际行为动作（如下订单）
    - ...

## Function Calling的工作原理

### OpenAI官方定义

OpenAI官方说明文档：https://platform.openai.com/docs/guides/function-calling

<img src="./function-calling-diagram-steps.png" width="600px"/>

### 描述Function Calling的另一个流程图

<img src="./function_calling.drawio.png"/>

### Calling是结果，理解和选择才是第一步

- 除了代表用户诉求的Prompt之外，Function Calling还需要将可用的工具信息（Function Definitions）也提供给模型
- 在第一次请求时，模型的核心工作如下：
    1. 理解Prompt所代表的“诉求”和Definitions所代表的“行动可能性”
    2. “选择”完成“诉求”所需要进行的“行动”（从“行动可能性”中获得）
    3. 根据所选择的“行动”，给出执行“行动”所需的“行动参数”（Parameters）
- 那么想一想：
    1. 什么影响“选择”的效果？
    2. 什么影响“行动”的可执行性和效果？

### 作为可选项的结果回调和最终回复输出

- 在对话流中，将Function Calling的结果（Function Result）与初始的Prompt诉求再次组合，提供给模型以获得最终的回复输出，是常见的流程（RAG就是一个典型的例子）
- 但如果我们将Function Calling用于非对话流场景，最终回复输出就不一定是必选项了，例如：
    1. 【只需要完成Calling动作】我们只是希望通过Function Calling完成行动选择和发起，接下来就进入业务处理流程，例如：理解用户表达并代替用户下单
    2. 【只需要完成行动参数Parameters生成】我们只是希望将Function Calling做好工具使用决策，并完成部分请求参数的生成，接下来需要走业务流程补全其他参数（比如鉴权信息），例如：敏感数据查询

在实际生产中，不给出最终回复输出，而只是使用Function Calling返回的调用方法数据，是很常见的用法。

## 实际调用Function Calling

### 从官方案例开始

#### 第一步：工具决策和调用信息生成

In [ ]:
import os
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

# 给出工具定义
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

# 给出诉求表达
messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

# 发起请求
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
# print(completion)
print(completion.choices[0].message.tool_calls[0].function)

- 情况1：问与工具的无关的问题会发生什么？
    - `messages = [{"role": "user", "content": "How are you today?"}]`
- 情况2：问无法获得确切行动参数的问题会发生什么？
    - `messages = [{"role": "user", "content": "What's the weather like today?"}]`

- 如何容错：

    ```python
    if completion.choices[0].message.tool_calls:
        print(completion.choices[0].message.tool_calls[0].function)
    else:
        print("No function is called.")
    ```

#### 第二步：实际调用工具

In [ ]:
function_calling_message = completion.choices[0].message
function_calling = completion.choices[0].message.tool_calls[0]

print("Call Function Name:", function_calling.function.name)
print("Call Function Arguments:", function_calling.function.arguments)

In [ ]:
import json

def get_weather(*, latitude:float, longitude:float):
    return {
        "temperature": 23,
        "weather": "Sunny",
        "wind_direction": "South",
        "windy": 2,
    }

functions = {
    "get_weather": get_weather
}

function_result = functions[function_calling.function.name](**json.loads(function_calling.function.arguments))
print(function_result)

#### 第三步：将结果返回给模型获取最终结果

In [21]:
print(messages)

[{'role': 'user', 'content': "What's the weather like in Paris today?"}]


In [ ]:
messages.append(function_calling_message)
messages.append({
    "role": "tool",
    "tool_call_id": function_calling.id,
    "content": str(function_result),
})

final_result = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(final_result.choices[0].message.content)

- 如果获得错误信息会怎么样？

In [ ]:
error_messages = messages[:1]
error_messages.append(function_calling_message)
error_messages.append({
    "role": "tool",
    "tool_call_id": function_calling.id,
    "content": str(TypeError("Key 'latitude' can not be supported any more, please use 'lat' instead.")),
})
print(error_messages)

final_result = client.chat.completions.create(
    model="gpt-4o",
    messages=error_messages,
    tools=tools,
)
print(final_result.choices[0])

- 答：会重试，但不多...

## 在实际应用场景中的一些案例

### 封装基本方法

In [ ]:
import os
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

import json
from typing import TypedDict
from openai import OpenAI

class FunctionCallingResult(TypedDict):
    name: str
    arguments: str

class ModelRequestWithFunctionCalling:
    def __init__(self):
        self._client = OpenAI(
            api_key=os.environ.get("OPENAI_API_KEY"),
            base_url=os.environ.get("OPENAI_BASE_URL"),
        )
        self._function_infos = {}
        self._function_mappings = {}
        self._messages = []
    
    def register_function(self, *, name, description, parameters, function, **kwargs):
        self._function_infos.update({
            name: {
                "type": "function",
                "function": {
                    "name": name,
                    "description": description,
                    "parameters": parameters,
                    **kwargs
                }
            }
        })
        self._function_mappings.update({ name: function })
        return self

    def reset_messages(self):
        self._messages = []
        return self
    
    def append_message(self, role, content, **kwargs):
        self._messages.append({ "role": role, "content": content, **kwargs })
        print("[Processing Messages]:", self._messages[-1])
        return self
    
    def _call(self, function_calling_result:FunctionCallingResult):
        function = self._function_mappings[function_calling_result.name]
        arguments = json.loads(function_calling_result.arguments)
        return function(**arguments)

    def request(self, *, role="user", content=None):
        if role and content:
            self._messages.append({ "role": role, "content": content })
        result = self._client.chat.completions.create(
            model="gpt-4o",
            messages=self._messages,
            tools=self._function_infos.values(),
        )
        self.append_message(**dict(result.choices[0].message))
        if result.choices[0].message.tool_calls:
            for tool_call in result.choices[0].message.tool_calls:
                call_result = self._call(tool_call.function)
                self.append_message("tool", str(call_result), tool_call_id=tool_call.id)
            return self.request()
        else:
            self.append_message("assistant", result.choices[0].message.content)
            return result.choices[0].message.content

### 联网检索现实场景

In [ ]:
import requests
import os
import json

amap_key = os.getenv("AMAP_KEY")
amap_base_url = os.getenv("AMAP_URL") # 默认是 https://restapi.amap.com/v5


def get_location_coordinate(location, city):
    url = f"{amap_base_url}/place/text?key={amap_key}&keywords={location}&region={city}"
    r = requests.get(url)
    result = r.json()
    if "pois" in result and result["pois"]:
        return result["pois"][0]
    return None


def search_nearby_pois(longitude, latitude, keyword):
    url = f"{amap_base_url}/place/around?key={amap_key}&keywords={keyword}&location={longitude},{latitude}"
    r = requests.get(url)
    result = r.json()
    ans = ""
    if "pois" in result and result["pois"]:
        for i in range(min(3, len(result["pois"]))):
            name = result["pois"][i]["name"]
            address = result["pois"][i]["address"]
            distance = result["pois"][i]["distance"]
            ans += f"{name}\n{address}\n距离：{distance}米\n\n"
    return ans

function_calling_request = ModelRequestWithFunctionCalling()

(
    function_calling_request
        .register_function(
            name="get_location_coordinate",
            description="根据POI名称，获得POI的经纬度坐标",
            parameters={
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "POI名称，必须是中文",
                    },
                    "city": {
                        "type": "string",
                        "description": "POI所在的城市名，必须是中文",
                    }
                },
                "required": ["location", "city"],
            },
            function=get_location_coordinate,
        )
        .register_function(
            name="search_nearby_pois",
            description="搜索给定坐标附近的poi",
            parameters={
                "type": "object",
                "properties": {
                    "longitude": {
                        "type": "string",
                        "description": "中心点的经度",
                    },
                    "latitude": {
                        "type": "string",
                        "description": "中心点的纬度",
                    },
                    "keyword": {
                        "type": "string",
                        "description": "目标poi的关键字",
                    }
                },
                "required": ["longitude", "latitude", "keyword"],
            },
            function=search_nearby_pois,
        )
)
result = function_calling_request.request(content="五道口附近的咖啡馆")
print("----------------------\n\n", result)

### 本地数据库查询

#### 数据准备

In [ ]:
import sqlite3

database_schema_string = """
CREATE TABLE orders (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    customer_id INT NOT NULL, -- 客户ID，不允许为空
    product_id STR NOT NULL, -- 产品ID，不允许为空
    price DECIMAL(10,2) NOT NULL, -- 价格，不允许为空
    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0代表待支付，1代表已支付，2代表已退款
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间
    pay_time TIMESTAMP -- 支付时间，可以为空
);
"""

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute(database_schema_string)

mock_data = [
    (1, 1001, 'TSHIRT_1', 50.00, 0, '2023-09-12 10:00:00', None),
    (2, 1001, 'TSHIRT_2', 75.50, 1, '2023-09-16 11:00:00', '2023-08-16 12:00:00'),
    (3, 1002, 'SHOES_X2', 25.25, 2, '2023-10-17 12:30:00', '2023-08-17 13:00:00'),
    (4, 1003, 'SHOES_X2', 25.25, 1, '2023-10-17 12:30:00', '2023-08-17 13:00:00'),
    (5, 1003, 'HAT_Z112', 60.75, 1, '2023-10-20 14:00:00', '2023-08-20 15:00:00'),
    (6, 1002, 'WATCH_X001', 90.00, 0, '2023-10-28 16:00:00', None)
]

for record in mock_data:
    cursor.execute('''
    INSERT INTO orders (id, customer_id, product_id, price, status, create_time, pay_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', record)

conn.commit()

def query_db(query):
    cursor.execute(query)
    return cursor.fetchall()

#### 调用执行

In [ ]:
function_calling_request = ModelRequestWithFunctionCalling()

(
    function_calling_request
        .register_function(
            name="query_db",
            description="使用此函数查询业务数据库获取结果，输出的SQL需要能够在Python的sqlite3中执行",
            parameters={
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                        SQL query extracting info to answer the user's question.
                        The query should be returned in plain text, not in JSON.
                        The query should only contain grammars supported by SQLite.
                        """,
                    }
                },
                "required": ["query"],
            },
            function=query_db,
        )
)

question = "2023年10月总共成交了几笔订单？"

result = function_calling_request.request(
    content=f"""
    问题：{ question },
    数据库元数据信息：{ database_schema_string },
"""
)

### 跨模型协作

#### 利用文心4.0以上模型作为搜索工具

##### 安装文心调用SDK

In [ ]:
%pip install erniebot

##### 测试

In [ ]:
import os
import erniebot

erniebot.api_type = "aistudio"
erniebot.access_token = os.environ.get("AISTUDIO_ACCESS_TOKEN")
# 访问aistudio.baidu.com注册账号，可获得自己的access_token

response = erniebot.ChatCompletion.create(
    model="ernie-4.0-turbo-8k",
    messages=[{
        "role": "user",
        "content": "最近英伟达GTC大会发布了哪些新闻？"
    }])

print(response.get_result())

##### 封装工具

In [ ]:
import erniebot

erniebot.api_type = "aistudio"
erniebot.access_token = os.environ.get("AISTUDIO_ACCESS_TOKEN")

def nl_search(question:str):
    prompt = f"""
基于联网搜索结果回答此问题：{ question }
其他输出要求：答案中的关键信息必须标注精确到内容页面的来源链接
你的回答：
"""
    response = erniebot.ChatCompletion.create(
    model="ernie-4.0",
    messages=[{
        "role": "user",
        "content": prompt,
    }])
    return response.get_result()

##### 调用执行

In [ ]:
function_calling_request = ModelRequestWithFunctionCalling()

(
    function_calling_request
        .register_function(
            name="nl_search",
            description="使用此工具，可以用自然语言输入，获得基于网络搜索的事实性结果总结",
            parameters={
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "使用自然语言总结用户关注的关键问题",
                    }
                },
                "required": ["question"],
            },
            function=nl_search,
        )
)

question = "今年英伟达GTC大会主要讲了哪些关键信息？"

result = function_calling_request.request(
    content=question,
)
print(result)

## Function Calling在大模型应用场景中带来的“质变”

- 知识层面：从模型自身知识（来源于训练语料）扩展到真实世界知识
- 行为层面：从“思考模拟器”、“问题应答”扩展到“理解问题-选择行动-发起请求-理解结果-给出回应”
- 架构层面：让模型不再是一个孤立模块，而是可以融入现有信息系统之中

给软件开发思想带来的冲击：

- 不是基于“规则”而是基于“世界理解”的调用
- 接纳没有明确的处理过程带来的输出不确定性（如数据查询）
- 不走极端：“全盘拒绝”和“全盘接受”都不可取